In [85]:
"""
Authors: Ronak Hegde and Vignesh Vasan
Run all cells in sequential order
"""

import silence_tensorflow.auto #silences all tensorflow warnings and logs
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.layers as layers
from tensorflow.keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import requests
import tensorflow as tf
import tensorflow_hub as hub
import random

In [70]:
def data_augmentation(train_dir):
    """ Increasing # of training samples with random transformations """
    datagen = image.ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40,
        width_shift_range = 0.3, 
        height_shift_range = 0.3,
        shear_range = 0.2, 
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode = 'nearest')
    
    generator = datagen.flow_from_directory(
        train_dir,
        target_size = (150,150),
        batch_size = 32, 
        class_mode = 'categorical'
    )
    return generator 

In [76]:
def data_augmentation(train_dir):
    """ Increasing # of training samples with random transformations """

    datagen = image.ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40,
        width_shift_range = 0.3, 
        height_shift_range = 0.3,
        shear_range = 0.2, 
        zoom_range = 0.2,
        horizontal_flip = True,
        fill_mode = 'nearest')
    
    generator = datagen.flow_from_directory(
        train_dir,
        target_size = (150,150),
        batch_size = 20,
        class_mode = 'categorical'
    )
    
    return generator 

In [89]:
train_dir = '../data/weather_dataset/train'
validation_dir = '../data/weather_dataset/validation'
test_dir = '../data/weather_dataset/test'

train_generator = data_augmentation(train_dir)

test_datagen = image.ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/4")
])
model.add(layers.Dense(4, activation = 'softmax'))

model.compile(
    loss = 'categorical_crossentropy', 
    optimizer = tf.keras.optimizers.RMSprop(lr=1e-4),
    metrics = ['acc']
)

Found 900 images belonging to 4 classes.
Found 112 images belonging to 4 classes.


In [90]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=20)

Epoch 1/30
40/40 [==============================] - 13s 317ms/step - loss: 4.0705 - acc: 0.2425 - val_loss: 2.5440 - val_acc: 0.3839
Epoch 2/30
40/40 [==============================] - 11s 267ms/step - loss: 3.1828 - acc: 0.3225
Epoch 3/30
40/40 [==============================] - 11s 275ms/step - loss: 2.3641 - acc: 0.4563
Epoch 4/30
40/40 [==============================] - 11s 280ms/step - loss: 1.8201 - acc: 0.5213
Epoch 5/30
40/40 [==============================] - 11s 272ms/step - loss: 1.4208 - acc: 0.6187
Epoch 6/30
40/40 [==============================] - 11s 272ms/step - loss: 1.2350 - acc: 0.6438
Epoch 7/30
40/40 [==============================] - 11s 267ms/step - loss: 1.0053 - acc: 0.7113
Epoch 8/30
40/40 [==============================] - 11s 280ms/step - loss: 0.8596 - acc: 0.7450
Epoch 9/30
40/40 [==============================] - 11s 268ms/step - loss: 0.7232 - acc: 0.7788
Epoch 10/30
40/40 [==============================] - 11s 264ms/step - loss: 0.6728 - acc: 0.7775
Ep

In [91]:
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size = (150,150),
    batch_size = 1,
    class_mode = 'categorical', 
#     shuffle = False
)

Found 113 images belonging to 4 classes.


In [93]:
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size = (150,150),
    batch_size = 1,
    class_mode = 'categorical', 
    shuffle = False
)

test_loss, test_acc = model.evaluate_generator(test_generator, steps = len(test_generator), verbose = True)
print('test acc:', test_acc)

Found 113 images belonging to 4 classes.
113/113 [==============================] - 4s 34ms/step - loss: 0.3928 - acc: 0.8496
test acc: 0.8495575189590454


In [101]:
def results_to_file(num_results, filename, test_generator):
    test_fnames = test_generator.filenames
    test_fnames = [os.path.join(test_dir, fname) for fname in test_fnames]
    
    y_true = test_generator.classes
    y_pred = model.predict_classes(test_generator)   
    
    assert len(y_true) == len(y_pred) == len(test_fnames), f"Incorrect number of Files, Predictions, True Values"
    
    default_mapping = test_generator.class_indices
    label_mapping = dict((v,k) for k, v in default_mapping.items())
    
    #pick (num_results) number of files from our test set to output 
    random_file_indices = random.sample(range(1,len(test_fnames)), num_results)
    
    with open(filename,'w') as f:
        for i in random_file_indices:
            image_path = test_fnames[i]
            predicted_label = label_mapping[y_pred[i]]
            true_label = label_mapping[y_true[i]]
            f.write(image_path + '\t' + 'predicted label: ' + predicted_label + '\t' + 'true label: ' + true_label + '\n')

In [102]:
results_to_file(25, 'results.txt', test_generator)